In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from calipertron import *

In [ ]:
# ADC sampling rate in Hz
sampling_rate = 251.748 * 1e3

raw = pd.read_csv("../frontend/100kHz_drive.txt", header = None, names = ["adc"])
raw['t'] = range(0, len(raw))
raw

In [ ]:
offset = 100000
n = 10000
d = raw.to_numpy()[offset:,][:n, 0]
sns.lineplot(x = range(n), y = d)
plt.show()

In [ ]:
# Apply a high-pass filter to remove low frequency noise
from scipy import signal

def apply_high_pass_filter(data, fs=100000, cutoff=1000, order=6):
    """
    Apply a high-pass filter to remove low frequency noise.
    
    Parameters:
    data (array-like): Input data to be filtered
    fs (int): Sampling frequency in Hz (default: 100000)
    cutoff (int): Cutoff frequency for high-pass filter in Hz (default: 1000)
    order (int): Filter order (default: 6)
    
    Returns:
    array-like: Filtered data
    """
    # Compute the Nyquist frequency
    nyquist = 0.5 * fs

    # Compute the normalized cutoff frequency
    normalized_cutoff = cutoff / nyquist

    # Design the high-pass filter
    b, a = signal.butter(order, normalized_cutoff, btype='high', analog=False)

    # Apply the filter to the data
    filtered_data = signal.filtfilt(b, a, data)
    
    return filtered_data

# Apply the high-pass filter to the data
d_filtered = apply_high_pass_filter(d)

# Define the number of points to plot
num_points = 1000

# Plot the original and filtered data for comparison using seaborn
plt.figure(figsize=(12, 6))
sns.lineplot(x=range(num_points), y=d[:num_points], label='Original')
sns.lineplot(x=range(num_points), y=d_filtered[:num_points], label='Filtered')
plt.xlabel('Sample')
plt.ylabel('ADC Value')
plt.title(f'Original vs High-Pass Filtered Data (First {num_points} Points)')
plt.legend()
plt.show()


In [ ]:
cutoff_freq = 800
pdm_signal = d_filtered
# Low-pass filter the PDM signal
nyquist = 0.5 * sampling_rate
b, a = signal.butter(4, cutoff_freq / nyquist, btype='low')
filtered_signal = signal.lfilter(b, a, pdm_signal)


filtered_signal = d_filtered.copy()
#filtered_signal[filtered_signal < 0] = 0
filtered_signal = filtered_signal[100:800,]
# Plot the filtered signal
plt.figure(figsize=(12, 6))
plt.plot(filtered_signal)
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Filtered PDM Signal')
plt.show()



In [ ]:
def find_zero_crossings(pdm_signal, sampling_rate, cutoff_freq):
    # Low-pass filter the PDM signal
    nyquist = 0.5 * sampling_rate
    b, a = signal.butter(4, cutoff_freq / nyquist, btype='low')
    filtered_signal = signal.lfilter(b, a, pdm_signal)
    
    # Interpolate the filtered signal
    t = np.arange(len(pdm_signal)) / sampling_rate
    t_interp = np.linspace(t[0], t[-1], len(pdm_signal) * 10)
    interp_signal = np.interp(t_interp, t, filtered_signal)
    
    # Detect zero crossings
    zero_crossings = np.where(np.diff(np.sign(interp_signal)))[0]
    
    return t_interp, interp_signal, t_interp[zero_crossings]

cutoff_freq = 700  # Hz

t_interp, interp_signal, zero_crossing_times = find_zero_crossings(d_filtered, sampling_rate, cutoff_freq)

# Plotting
plt.figure(figsize=(12, 8))

# Plot original PDM signal
plt.subplot(3, 1, 1)
#plt.step(t_interp, d_filtered, where='post', label='PDM Signal')
plt.title('Original PDM Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

# Plot filtered and interpolated signal
plt.subplot(3, 1, 2)
plt.plot(t_interp, interp_signal, label='Filtered Signal')
plt.title('Filtered and Interpolated Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

# Plot filtered signal with zero crossings
plt.subplot(3, 1, 3)
plt.plot(t_interp, interp_signal, label='Filtered Signal')
plt.plot(zero_crossing_times, np.zeros_like(zero_crossing_times), 'ro', label='Zero Crossings')
plt.title('Filtered Signal with Zero Crossings')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

plt.tight_layout()
plt.show()

print("Number of zero crossings:", len(zero_crossing_times))
print("First few zero crossing times:", zero_crossing_times[:5])